In [1]:
# importing python modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
from anndata import read_h5ad
from anndata import read_csv
import anndata
from plotnine import * 
from plotnine.data import mtcars
import venn
from scipy import stats
import seaborn as sns
import math
from scipy.stats import pearsonr
from sklearn.metrics import roc_curve, auc, roc_auc_score
import abc
import batchglm.api as glm
import logging
import patsy
import glob
from random import sample
import scipy.sparse
from typing import Union, Dict, Tuple, List, Set
import math
from anndata import AnnData
from typing import Optional, Union, Mapping  # Special
from typing import Sequence, Collection, Iterable  # ABCs
_VarNames = Union[str, Sequence[str]]

%matplotlib inline
sc.logging.print_header()

scanpy==1.8.1 anndata==0.7.5 umap==0.5.1 numpy==1.21.0 scipy==1.6.2 pandas==1.2.3 scikit-learn==0.23.2 statsmodels==0.12.2 python-igraph==0.9.1 louvain==0.7.0 leidenalg==0.8.2 pynndescent==0.5.2


# Anndata to DF function

In [2]:
from pandas.api.types import is_categorical_dtype

In [3]:
def sanitize_anndata(adata):
    adata._sanitize()

In [4]:
def sc_prepare_dataframe(
    adata: AnnData,
    var_names: Union[_VarNames, Mapping[str, _VarNames]],
    groupby: Optional[str] = None,
    use_raw: Optional[bool] = None,
    log: bool = False,
    num_categories: int = 7,
    layer=None,
    gene_symbols: Optional[str] = None,
):
    """
    Given the anndata object, prepares a data frame in which the row index are the categories
    defined by group by and the columns correspond to var_names.
    Parameters
    ----------
    adata
        Annotated data matrix.
    var_names
        `var_names` should be a valid subset of  `adata.var_names`.
    groupby
        The key of the observation grouping to consider. It is expected that
        groupby is a categorical. If groupby is not a categorical observation,
        it would be subdivided into `num_categories`.
    use_raw
        Use `raw` attribute of `adata` if present.
    log
        Use the log of the values
    num_categories
        Only used if groupby observation is not categorical. This value
        determines the number of groups into which the groupby observation
        should be subdivided.
    gene_symbols
        Key for field in .var that stores gene symbols.
    Returns
    -------
    Tuple of `pandas.DataFrame` and list of categories.
    """
    from scipy.sparse import issparse

    sanitize_anndata(adata)
    if use_raw is None and adata.raw is not None:
        use_raw = True
    if isinstance(var_names, str):
        var_names = [var_names]

    if groupby is not None:
        if groupby not in adata.obs_keys():
            raise ValueError(
                'groupby has to be a valid observation. '
                f'Given {groupby}, valid observations: {adata.obs_keys()}'
            )

    if gene_symbols is not None and gene_symbols in adata.var.columns:
        # translate gene_symbols to var_names
        # slow method but gives a meaningful error if no gene symbol is found:
        translated_var_names = []
        for symbol in var_names:
            if symbol not in adata.var[gene_symbols].values:
                logg.error(
                    f"Gene symbol {symbol!r} not found in given "
                    f"gene_symbols column: {gene_symbols!r}"
                )
                return
            translated_var_names.append(
                adata.var[adata.var[gene_symbols] == symbol].index[0]
            )
        symbols = var_names
        var_names = translated_var_names
    if layer is not None:
        if layer not in adata.layers.keys():
            raise KeyError(
                f'Selected layer: {layer} is not in the layers list. '
                f'The list of valid layers is: {adata.layers.keys()}'
            )
        matrix = adata[:, var_names].layers[layer]
    elif use_raw:
        matrix = adata.raw[:, var_names].X
    else:
        matrix = adata[:, var_names].X

    if issparse(matrix):
        matrix = matrix.toarray()
    if log:
        matrix = np.log1p(matrix)

    obs_tidy = pd.DataFrame(matrix, columns=var_names)
    if groupby is None:
        groupby = ''
        categorical = pd.Series(np.repeat('', len(obs_tidy))).astype('category')
    else:
        if not is_categorical_dtype(adata.obs[groupby]):
            # if the groupby column is not categorical, turn it into one
            # by subdividing into  `num_categories` categories
            categorical = pd.cut(adata.obs[groupby], num_categories)
        else:
            categorical = adata.obs[groupby]

    obs_tidy.set_index(categorical, groupby, inplace=True)
    if gene_symbols is not None:
        # translate the column names to the symbol names
        obs_tidy.rename(
            columns=dict([(var_names[x], symbols[x]) for x in range(len(var_names))]),
            inplace=True,
        )
    categories = obs_tidy.index.categories

    return categories, obs_tidy


# making files for panel A heatmap

In [5]:
adata_processed = sc.read_h5ad('SS2_processed.h5ad')

In [6]:
adata_processed_tumor = adata_processed[adata_processed.obs['sort'] == 'Tumor'].copy()
adata_processed_met = adata_processed[adata_processed.obs['sort'] == 'Metastatic'].copy()

In [7]:
final_met_genes = pd.read_csv('SS2_met_vs_primary_met_genes_list.csv')
final_tumor_genes = pd.read_csv('SS2_met_vs_primary_tumor_genes_list.csv')

In [8]:
tumor_mean_expression_df = sc_prepare_dataframe(adata_processed_tumor, [x for x in final_tumor_genes['global'].tolist() if str(x) != 'nan']+[x for x in final_met_genes['global'].tolist() if str(x) != 'nan'], groupby='Tumor_ID',use_raw=False)

In [9]:
tumor_mean_expression_df = tumor_mean_expression_df[1].groupby(level=0).mean()
tumor_mean_expression_df.index = [i+'_Tumor' for i in tumor_mean_expression_df.index]
tumor_mean_expression_df = tumor_mean_expression_df.T
tumor_mean_expression_df

,H3204_Tumor,H4272_Tumor,H5097_Tumor,H5471_Tumor,HCI001_Tumor,HCI005_Tumor,HCI009_Tumor,HCI010_Tumor,HCI011_Tumor,J2036_Tumor,J53353_Tumor,J55454_Tumor
EEF1A1P5,0.717580,-0.290372,-0.073775,0.513669,-0.344379,0.186287,-0.548027,-0.153084,-0.060069,-0.412355,-0.303884,-0.545571
RPL30,-0.283744,-0.805813,0.071603,-0.439677,-0.671816,-0.016656,-1.031574,0.306881,0.247110,1.734200,0.537392,1.488159
ANGPTL4,-0.356391,0.496628,-0.105378,0.069880,-0.174617,-0.138322,1.969211,0.368243,-0.192943,-0.043230,0.342073,-0.302559
PRSS23,-0.232280,-0.224729,-0.347755,-0.339719,-0.289547,1.846410,-0.336749,-0.151037,0.058988,-0.361712,-0.311427,-0.316849
TGFBI,-0.142452,0.800384,-0.102300,-0.126295,-0.165573,-0.174173,-0.143050,1.235087,-0.146260,-0.151022,-0.104774,0.026906
...,...,...,...,...,...,...,...,...,...,...,...,...
MYL12B,-0.173553,0.154036,0.674665,-0.351421,0.289511,-0.326166,-1.192217,-0.242682,0.293396,0.353074,-0.530506,-0.420215
GDPD3,-0.408422,-0.405251,-0.258351,-0.378908,-0.286903,0.491827,-0.252357,0.046683,0.497920,-0.408001,-0.421529,-0.159331
GGCT,-0.347583,0.095099,0.294636,-0.163911,0.491966,0.502934,0.234501,-0.459007,-0.125028,-0.412960,-0.365906,-0.752320
TSPYL1,0.116197,-0.182932,0.955185,-0.570155,-0.183526,0.310678,-0.579097,0.209288,0.002950,-0.774271,-0.178202,-0.559603


In [10]:
met_mean_expression_df = sc_prepare_dataframe(adata_processed_met, [x for x in final_tumor_genes['global'].tolist() if str(x) != 'nan']+[x for x in final_met_genes['global'].tolist() if str(x) != 'nan'], groupby='Tumor_ID',use_raw=False)

In [11]:
met_mean_expression_df = met_mean_expression_df[1].groupby(level=0).mean()
met_mean_expression_df.index = [i+'_Metastatic' for i in met_mean_expression_df.index]
met_mean_expression_df = met_mean_expression_df.T
met_mean_expression_df

,H3204_Metastatic,H4272_Metastatic,H5097_Metastatic,H5471_Metastatic,HCI001_Metastatic,HCI005_Metastatic,HCI009_Metastatic,HCI010_Metastatic,HCI011_Metastatic,J2036_Metastatic,J53353_Metastatic,J55454_Metastatic
EEF1A1P5,0.133296,-0.429847,0.138993,-0.575557,-0.396568,0.060406,-0.402854,-0.131433,0.072155,0.345089,-0.323110,0.285588
RPL30,-0.452242,-1.017561,-0.096528,0.596625,-0.271178,0.013234,-0.628567,0.210265,-0.220210,0.708033,0.383551,0.495561
ANGPTL4,-0.319142,-0.342971,0.004992,0.502590,-0.140590,-0.153501,-0.044153,0.239189,0.027410,0.048719,-0.189236,-0.345673
PRSS23,-0.349123,-0.130735,-0.404662,-0.299685,-0.290067,1.090807,-0.181497,-0.173166,-0.073524,-0.373948,-0.304811,-0.115951
TGFBI,-0.170790,0.158170,-0.163614,-0.111877,-0.153397,-0.162906,0.109206,-0.196092,-0.174224,-0.150634,-0.177024,-0.125318
...,...,...,...,...,...,...,...,...,...,...,...,...
MYL12B,-0.106255,0.576927,0.992392,0.136495,-0.075477,-0.186469,-0.026344,-0.170247,0.728944,0.455178,-0.669130,-1.043709
GDPD3,-0.410025,-0.341050,0.154203,-0.386880,-0.392942,0.941159,0.478934,0.159136,1.980593,-0.408744,-0.418416,0.073946
GGCT,-0.142158,0.094664,0.397560,-0.946969,0.361135,0.708871,1.308439,-0.340700,-0.087159,-0.137463,-0.618078,-0.207160
TSPYL1,0.133797,-0.254688,1.035941,-0.561507,-0.151323,0.515852,-0.591726,0.346526,0.210587,-0.413025,-0.420504,0.195281


In [12]:
global_mean_expression_df = sc_prepare_dataframe(adata_processed, [x for x in final_tumor_genes['global'].tolist() if str(x) != 'nan']+[x for x in final_met_genes['global'].tolist() if str(x) != 'nan'], groupby='sort',use_raw=False)


In [13]:
global_mean_expression_df = global_mean_expression_df[1].groupby(level=0).mean()
global_mean_expression_df.index = ['global_'+i for i in global_mean_expression_df.index]
global_mean_expression_df = global_mean_expression_df.T
global_mean_expression_df

,global_Metastatic,global_Tumor
EEF1A1P5,-0.076863,0.038294
RPL30,-0.154724,0.077085
ANGPTL4,-0.073875,0.036805
PRSS23,-0.093475,0.046570
TGFBI,-0.117709,0.052614
...,...,...
MYL12B,0.210119,-0.104683
GDPD3,0.217623,-0.108421
GGCT,0.161755,-0.080587
TSPYL1,0.176180,-0.087774


In [14]:
combined_df_global_DEGs_mean_expression = pd.concat([tumor_mean_expression_df,met_mean_expression_df,global_mean_expression_df],axis=1)
combined_df_global_DEGs_mean_expression

,H3204_Tumor,H4272_Tumor,H5097_Tumor,H5471_Tumor,HCI001_Tumor,HCI005_Tumor,HCI009_Tumor,HCI010_Tumor,HCI011_Tumor,J2036_Tumor,...,HCI001_Metastatic,HCI005_Metastatic,HCI009_Metastatic,HCI010_Metastatic,HCI011_Metastatic,J2036_Metastatic,J53353_Metastatic,J55454_Metastatic,global_Metastatic,global_Tumor
EEF1A1P5,0.717580,-0.290372,-0.073775,0.513669,-0.344379,0.186287,-0.548027,-0.153084,-0.060069,-0.412355,...,-0.396568,0.060406,-0.402854,-0.131433,0.072155,0.345089,-0.323110,0.285588,-0.076863,0.038294
RPL30,-0.283744,-0.805813,0.071603,-0.439677,-0.671816,-0.016656,-1.031574,0.306881,0.247110,1.734200,...,-0.271178,0.013234,-0.628567,0.210265,-0.220210,0.708033,0.383551,0.495561,-0.154724,0.077085
ANGPTL4,-0.356391,0.496628,-0.105378,0.069880,-0.174617,-0.138322,1.969211,0.368243,-0.192943,-0.043230,...,-0.140590,-0.153501,-0.044153,0.239189,0.027410,0.048719,-0.189236,-0.345673,-0.073875,0.036805
PRSS23,-0.232280,-0.224729,-0.347755,-0.339719,-0.289547,1.846410,-0.336749,-0.151037,0.058988,-0.361712,...,-0.290067,1.090807,-0.181497,-0.173166,-0.073524,-0.373948,-0.304811,-0.115951,-0.093475,0.046570
TGFBI,-0.142452,0.800384,-0.102300,-0.126295,-0.165573,-0.174173,-0.143050,1.235087,-0.146260,-0.151022,...,-0.153397,-0.162906,0.109206,-0.196092,-0.174224,-0.150634,-0.177024,-0.125318,-0.117709,0.052614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MYL12B,-0.173553,0.154036,0.674665,-0.351421,0.289511,-0.326166,-1.192217,-0.242682,0.293396,0.353074,...,-0.075477,-0.186469,-0.026344,-0.170247,0.728944,0.455178,-0.669130,-1.043709,0.210119,-0.104683
GDPD3,-0.408422,-0.405251,-0.258351,-0.378908,-0.286903,0.491827,-0.252357,0.046683,0.497920,-0.408001,...,-0.392942,0.941159,0.478934,0.159136,1.980593,-0.408744,-0.418416,0.073946,0.217623,-0.108421
GGCT,-0.347583,0.095099,0.294636,-0.163911,0.491966,0.502934,0.234501,-0.459007,-0.125028,-0.412960,...,0.361135,0.708871,1.308439,-0.340700,-0.087159,-0.137463,-0.618078,-0.207160,0.161755,-0.080587
TSPYL1,0.116197,-0.182932,0.955185,-0.570155,-0.183526,0.310678,-0.579097,0.209288,0.002950,-0.774271,...,-0.151323,0.515852,-0.591726,0.346526,0.210587,-0.413025,-0.420504,0.195281,0.176180,-0.087774


In [15]:
new_order = ['J55454_Tumor','H5471_Tumor','HCI005_Tumor','H3204_Tumor','H4272_Tumor','HCI009_Tumor',
            'HCI011_Tumor','HCI001_Tumor','H5097_Tumor','J2036_Tumor','J53353_Tumor','HCI010_Tumor','global_Tumor',
            'J55454_Metastatic','H5471_Metastatic','HCI005_Metastatic','H3204_Metastatic','H4272_Metastatic','HCI009_Metastatic',
            'HCI011_Metastatic','HCI001_Metastatic','H5097_Metastatic','J2036_Metastatic','J53353_Metastatic','HCI010_Metastatic','global_Metastatic']

In [16]:
combined_df_global_DEGs_mean_expression = combined_df_global_DEGs_mean_expression[new_order]
combined_df_global_DEGs_mean_expression

,J55454_Tumor,H5471_Tumor,HCI005_Tumor,H3204_Tumor,H4272_Tumor,HCI009_Tumor,HCI011_Tumor,HCI001_Tumor,H5097_Tumor,J2036_Tumor,...,H3204_Metastatic,H4272_Metastatic,HCI009_Metastatic,HCI011_Metastatic,HCI001_Metastatic,H5097_Metastatic,J2036_Metastatic,J53353_Metastatic,HCI010_Metastatic,global_Metastatic
EEF1A1P5,-0.545571,0.513669,0.186287,0.717580,-0.290372,-0.548027,-0.060069,-0.344379,-0.073775,-0.412355,...,0.133296,-0.429847,-0.402854,0.072155,-0.396568,0.138993,0.345089,-0.323110,-0.131433,-0.076863
RPL30,1.488159,-0.439677,-0.016656,-0.283744,-0.805813,-1.031574,0.247110,-0.671816,0.071603,1.734200,...,-0.452242,-1.017561,-0.628567,-0.220210,-0.271178,-0.096528,0.708033,0.383551,0.210265,-0.154724
ANGPTL4,-0.302559,0.069880,-0.138322,-0.356391,0.496628,1.969211,-0.192943,-0.174617,-0.105378,-0.043230,...,-0.319142,-0.342971,-0.044153,0.027410,-0.140590,0.004992,0.048719,-0.189236,0.239189,-0.073875
PRSS23,-0.316849,-0.339719,1.846410,-0.232280,-0.224729,-0.336749,0.058988,-0.289547,-0.347755,-0.361712,...,-0.349123,-0.130735,-0.181497,-0.073524,-0.290067,-0.404662,-0.373948,-0.304811,-0.173166,-0.093475
TGFBI,0.026906,-0.126295,-0.174173,-0.142452,0.800384,-0.143050,-0.146260,-0.165573,-0.102300,-0.151022,...,-0.170790,0.158170,0.109206,-0.174224,-0.153397,-0.163614,-0.150634,-0.177024,-0.196092,-0.117709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MYL12B,-0.420215,-0.351421,-0.326166,-0.173553,0.154036,-1.192217,0.293396,0.289511,0.674665,0.353074,...,-0.106255,0.576927,-0.026344,0.728944,-0.075477,0.992392,0.455178,-0.669130,-0.170247,0.210119
GDPD3,-0.159331,-0.378908,0.491827,-0.408422,-0.405251,-0.252357,0.497920,-0.286903,-0.258351,-0.408001,...,-0.410025,-0.341050,0.478934,1.980593,-0.392942,0.154203,-0.408744,-0.418416,0.159136,0.217623
GGCT,-0.752320,-0.163911,0.502934,-0.347583,0.095099,0.234501,-0.125028,0.491966,0.294636,-0.412960,...,-0.142158,0.094664,1.308439,-0.087159,0.361135,0.397560,-0.137463,-0.618078,-0.340700,0.161755
TSPYL1,-0.559603,-0.570155,0.310678,0.116197,-0.182932,-0.579097,0.002950,-0.183526,0.955185,-0.774271,...,0.133797,-0.254688,-0.591726,0.210587,-0.151323,1.035941,-0.413025,-0.420504,0.346526,0.176180


In [17]:
combined_df_global_DEGs_mean_expression.to_csv('global_met_vs_tumor_mean_expression_gc.csv')

In [18]:
low_met_tumors = ['J55454', 'H5471', 'HCI005', 'H3204','H4272']
intermediate_met_tumors = ['HCI009', 'HCI011', 'HCI001']
high_met_tumors = ['H5097', 'J2036', 'J53353', 'HCI010']

In [19]:
metadata = pd.DataFrame(index=new_order)
metadata['Tumor_ID'] = [i.split('_')[0] for i in metadata.index]
metadata['sort'] = [i.split('_')[1] for i in metadata.index]
for i in metadata.index:
    tumor_id = metadata.loc[i, 'Tumor_ID']
    if tumor_id in low_met_tumors:
        metadata.loc[i, 'metastatic_potential_group'] = 'low'
    elif tumor_id in intermediate_met_tumors+['H5471']:
        metadata.loc[i, 'metastatic_potential_group'] = 'moderate'
    elif tumor_id in high_met_tumors:
        metadata.loc[i, 'metastatic_potential_group'] = 'high'
    else:
        metadata.loc[i, 'metastatic_potential_group'] = 'all'
metadata

,Tumor_ID,sort,metastatic_potential_group
J55454_Tumor,J55454,Tumor,low
H5471_Tumor,H5471,Tumor,low
HCI005_Tumor,HCI005,Tumor,low
H3204_Tumor,H3204,Tumor,low
H4272_Tumor,H4272,Tumor,low
HCI009_Tumor,HCI009,Tumor,moderate
HCI011_Tumor,HCI011,Tumor,moderate
HCI001_Tumor,HCI001,Tumor,moderate
H5097_Tumor,H5097,Tumor,high
J2036_Tumor,J2036,Tumor,high


In [20]:
metadata.to_csv('global_met_vs_tumor_mean_expression_metadata.csv')

In [21]:
gene_metadata = pd.DataFrame()
gene_metadata['gene'] = [x for x in final_tumor_genes['global'].tolist() if str(x) != 'nan']+[x for x in final_met_genes['global'].tolist() if str(x) != 'nan']
gene_metadata = gene_metadata.set_index('gene')
for i in gene_metadata.index:
    if i in [x for x in final_tumor_genes['global'].tolist() if str(x) != 'nan']:
        gene_metadata.loc[i, 'group'] = 'tumor'
    elif i in [x for x in final_met_genes['global'].tolist() if str(x) != 'nan']:
        gene_metadata.loc[i, 'group'] = 'met'
        
gene_metadata.to_csv('global_met_vs_tumor_mean_expression_gene_metadata.csv')